In [1]:
import rospy
from geometry_msgs.msg import Twist, Point, Quaternion
from math import radians, copysign, sqrt, pow, pi, atan2
import time
from threading import Thread
from nav_msgs.msg import Odometry

In [2]:
%run ContinuousMovement_HelperFunctions.ipynb

In [3]:
# Create Node to communicate with ROS
rospy.init_node('sub_odom')
dx,dy,angle = 0,0,0
real_angle, real_x, real_y = 0,0,0
moved_distance = 0

In [4]:
def callback(data):
    global dx,dy,angle
    dx = data.pose.pose.position.x
    dy = data.pose.pose.position.y
    
    quat = [data.pose.pose.orientation.x,
            data.pose.pose.orientation.y,
            data.pose.pose.orientation.z,
            data.pose.pose.orientation.w]
    
    angle = math.degrees(euler_from_quaternion(quat))

In [5]:
def measurementThread():
    running_thread = True
    
    # Create a suscribe to Odometry node using callback function   
    odomSuscriber = rospy.Subscriber('/odom', Odometry, callback)
    
    rate = rospy.Rate(1)
    rate.sleep()
    rospy.spin()

In [6]:
def calculateDistance(x0,y0,x1,y1):
    return math.sqrt((x0-x1)**2+(y0-y1)**2)

In [7]:
def Move(distance=1,speed=0.2):
    direction = -1 if speed < 0 else 1
    global dx,dy,angle
    global real_angle, real_x, real_y
    global moved_distance
    
    move = Twist()
    move.linear.x=speed
    r = rospy.Rate(2)
    
    init_x = dx
    init_y = dy
    
    while calculateDistance(init_x,init_y,dx,dy) < distance*0.95:
        pub.publish(move)
        time.sleep(0.8)
    moved_distance = calculateDistance(init_x,init_y,dx,dy)*direction
    print('/nmoved: ', moved_distance)
    #print(f'shitty coordinates: angle={angle}, y={dx}, x={dy}')

In [8]:
def Rotate(degrees=10, direction='left'):
    global dx,dy,angle
    global angle
    global real_angle
    
    # Angle symbol
    direct = 1 if direction=='left' else -1
    
    # Create msg to communicate with ROS
    rotate = Twist()
    
    # Define speed and direction of rotation
    rotate.angular.z = direct*0.175
    r = rospy.Rate(2)

    # Rotate and calculate how much it rotated
    init_angle = angle
    while abs(init_angle-angle) < degrees*0.9:
        pub.publish(rotate)
        time.sleep(0.8)
        
    # Update real_angle
    rotated=direct*abs(init_angle-angle)
    real_angle-=rotated
    
    # Make sure real_angle is between 0-360
    if real_angle/360>1:
        real_angle-=360
    elif real_angle<0:
        real_angle+=360
    
    print('rotated: ', rotated)
    #print(f'shitty coordinates: angle={angle}, y={dx}, x={dy}')

In [9]:
def update_coordinates():
    
    time.sleep(0.5)
    
    global moved_distance
    global real_angle, real_x, real_y
    
    if real_angle == 0: # Moved Forward ==> Y++
        real_y+=moved_distance
        
    elif real_angle == 90: # Moved Right ==> X++
        real_x+=moved_distance
        
    elif real_angle == 180: # Moved Backward ==> Y--
        real_y-=moved_distance
        
    elif real_angle == 270: # Moved Left ==> X--
        real_x-=moved_distance
        
    else:  # ==> Calculate X,Y components 
        delta_x = moved_distance*math.sin(math.radians(real_angle))
        delta_y = moved_distance*math.cos(math.radians(real_angle))
               
        real_x += delta_x
        real_y += delta_y

In [10]:
# Create a publisher to control movement
pub = rospy.Publisher('cmd_vel', Twist, queue_size=1)

# Run Thread
running_thread = False
if not running_thread:
    thread = Thread(target = measurementThread)
    thread.start()

In [57]:
Move(distance=0.05, speed=+0.15)  # meters, m/s
update_coordinates()
print(f'new coordinates: angle={real_angle}, y={real_y}, x={real_x}')

/nmoved:  0.07572264257469076
new coordinates: angle=359.6503217443994, y=4.437208512665624, x=1.1578500598759678


In [54]:
Rotate(degrees=10, direction='left')
print(f'new coordinates: angle={real_angle}, y={real_y}, x={real_x}')

rotated:  10.289324826243899
new coordinates: angle=359.6503217443994, y=4.0595471853451555, x=1.1601549695029412
